In [1]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.log_device_placement = True

/usr/lib64/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# read csv

In [3]:
filename_queue = tf.train.string_input_producer(["data/1.csv"], num_epochs=1)

reader = tf.TextLineReader()

filepath, value = reader.read(filename_queue)

# set default value for different data type
record_defaults = [[''] for _ in xrange(2)]

tensor_list = tf.decode_csv(value, record_defaults=record_defaults, field_delim='\001')
with tf.Session(config=config) as sess:
    sess.run(tf.local_variables_initializer())
    tf.train.start_queue_runners()
    num_examples = 0
    try:
        while True:
            s_key, s_value = sess.run([filepath, value])
            print( s_key, s_value)
            num_examples += 1
    except tf.errors.OutOfRangeError:
        pass
    print ("There are", num_examples, "examples")

('data/1.csv:1', '1,lzx')
('data/1.csv:2', '3,wzl')
('data/1.csv:3', '2,carl')
('There are', 3, 'examples')


# read TFRecord

In [2]:
reader = tf.TFRecordReader()

filenames = ['/data01/user/caobaiyu/tensorflow/krRec/training.tfrecords']
filequeue = tf.train.string_input_producer(filenames, shuffle=True)

key, value = reader.read(filequeue)
features = tf.parse_single_example(value,
            features = {
            'examp1': tf.FixedLenFeature([400], tf.float32),
            'examp2': tf.FixedLenFeature([400], tf.float32)
            })
sample1 = features['examp1']
sample2 = features['examp2']

examp1Batch, examp2Batch = tf.train.shuffle_batch([sample1, sample2], 
                                                  batch_size=1,
                                                  capacity=10, 
                                                  min_after_dequeue=5)

In [3]:
%%time
with tf.Session(config=config) as sess:
    sess.run(tf.local_variables_initializer())
    tf.train.start_queue_runners()
    num_examples = 0
    try:
        while num_examples < 541790:
            s1, s2 = sess.run([sample1, sample2])
            num_examples += 1
    except:
       pass
    print ("There are", num_examples, "examples")

('There are', 541790, 'examples')
CPU times: user 6min 42s, sys: 59 s, total: 7min 41sERROR:tensorflow:Exception in QueueRunner: Enqueue operation was cancelled
	 [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer, input_producer/RandomShuffle)]]

Wall time: 3min 17s
ERROR:tensorflow:Exception in QueueRunner: Enqueue operation was cancelled
	 [[Node: shuffle_batch/random_shuffle_queue_enqueue = QueueEnqueueV2[Tcomponents=[DT_FLOAT, DT_FLOAT], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](shuffle_batch/random_shuffle_queue, ParseSingleExample/Squeeze_examp1/_7, ParseSingleExample/Squeeze_examp2/_9)]]


Exception in thread QueueRunnerThread-input_producer-input_producer/input_producer_EnqueueMany:
Traceback (most recent call last):
  File "/usr/lib64/python2.7/threading.py", line 811, in __bootstrap_inner
    self.run()
  File "/usr/lib64/python2.7/threading.py", line 764, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/usr/lib/python2.7/site-packages/tensorflow/python/training/queue_runner_impl.py", line 238, in _run
    enqueue_callable()
  File "/usr/lib/python2.7/site-packages/tensorflow/python/client/session.py", line 1231, in _single_operation_run
    target_list_as_strings, status, None)
  File "/usr/lib/python2.7/site-packages/tensorflow/python/framework/errors_impl.py", line 473, in __exit__
    c_api.TF_GetCode(self.status.status))
CancelledError: Enqueue operation was cancelled
	 [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_produce